In [30]:
# !/usr/bin/python
# -*- coding: utf-8 -*-
# Author: Selvaria
# 完全使用selenium获取企业信息存入dic_company_detail以及dic_company表

import requests
import time
from bs4 import BeautifulSoup
import csv
import pandas
import random
from selenium import webdriver
import json
import pymysql


  
def get_login(): #利用selenium手动登录,获取cookies
    url = 'https://www.qichacha.com/user_login'
#         url = 'https://www.qichacha.com'
    browser = webdriver.Chrome() 
    browser.get(url) 
    #login_click = browser.find_element_by_xpath('/html/body/header/div/ul[2]/li[7]/a')  #登陆
    #browser.execute_script("arguments[0].click();", login_click)
    time.sleep(40) #登录并录入验证码  #15201760096 123456  #18698925437 #13342264161 
    req = requests.Session()
    cookies = browser.get_cookies()
    
    req = requests.Session()
    for cookie in cookies:
        req.cookies.set(cookie['name'],cookie['value'])
    return cookies

cookie = get_login()
print(type(cookie))
data = json.dumps(cookie)
with open('qichacha_cookies.txt', 'w') as f:    
    f.write(data)
print(data)

<class 'list'>
[{"domain": "www.qichacha.com", "expiry": 1576875723, "httpOnly": false, "name": "hasShow", "path": "/", "secure": false, "value": "1"}, {"domain": ".qichacha.com", "expiry": 1608368523, "httpOnly": false, "name": "Hm_lvt_3456bee468c83cc63fb5147f119f1075", "path": "/", "secure": false, "value": "1576832512"}, {"domain": ".qichacha.com", "httpOnly": false, "name": "Hm_lpvt_3456bee468c83cc63fb5147f119f1075", "path": "/", "secure": false, "value": "1576832524"}, {"domain": ".qichacha.com", "expiry": 1608368523, "httpOnly": false, "name": "zg_de1d1a35bfa24ce29bbf2c7eb17e6c4f", "path": "/", "secure": false, "value": "%7B%22sid%22%3A%201576832511207%2C%22updated%22%3A%201576832523896%2C%22info%22%3A%201576832511210%2C%22superProperty%22%3A%20%22%7B%7D%22%2C%22platform%22%3A%20%22%7B%7D%22%2C%22utm%22%3A%20%22%7B%7D%22%2C%22referrerDomain%22%3A%20%22%22%2C%22cuid%22%3A%20%22754ac4a9a0832e8e97b2029530661b56%22%7D"}, {"domain": ".qichacha.com", "expiry": 1608368523, "httpOnly": f

In [29]:
#主程序
import requests
import time
from bs4 import BeautifulSoup
import csv
import pandas
import random
from selenium import webdriver
import json
import pymysql


class QichachaSelenium(object):

    def get_cookies(self):
        f= open('qichacha_cookies.txt')
        data_json = f.read()
        data_handle = json.loads(data_json)
        return data_handle

    def get_company_info(self, name, cookies): 
        #name = '金钻石油机械股份有限公司'

        url_search = 'https://www.qichacha.com/search?key=%s' %name #目标网址
        browser = webdriver.Chrome()
        url_l= 'https://www.qichacha.com/user_login' #这一步特别重要，必须先打开一个目标网址对应的界面
        browser.get(url_l) 
        for cookie_item in cookies:
            #print({'name': cookie_item, 'value': cookies[cookie_item]})
            browser.add_cookie(cookie_item)
        browser.get(url_search)

        try:
            num = browser.find_element_by_xpath('//*[@id="countOld"]/span').text
        except Exception as e:
            print('基本被封，请手动查询')
            print(str(e))
            #return None

        if num.strip() == '0':
            print('无此企业', name)
            return 0
        else:
            time.sleep(1)
            search_url = browser.find_element_by_xpath('//*[@id="search-result"]/tr[1]/td[3]/a').get_attribute("href") 
            url_c = 'https://www.qichacha.com%s' %'/cbase'+search_url[29:-5]
            #print(url_c)
            #com_web_q = 'https://www.qichacha.com' + search_url #企查查的企业主页
            url_company = browser.get(url_c) 
            time.sleep(1)
            button = browser.find_element_by_xpath('//*[@id="firstepdadModal"]/div/div/div[2]/button')
            browser.execute_script("arguments[0].click();", button)
            try:
                time.sleep(1)
                #table =  browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody').text
                #print(table)
                company_title = browser.find_element_by_xpath('//*[@id="company-top"]/div[2]/div[2]/div[1]/h1').text #企业名
                company_lead = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[1]/td[2]/div/div/div[2]/a[1]/h2').text #法人
                company_tel = browser.find_element_by_xpath('//*[@id="company-top"]/div[2]/div[2]/div[3]/div[1]/span[1]/span[2]').text.strip()
                company_email = browser.find_element_by_xpath('//*[@id="company-top"]/div[2]/div[2]/div[3]/div[2]/span[1]/span[2]').text
                #print(company_lead, company_tel, company_email)

                found_cash = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[1]/td[4]').text.strip() #注册资本
                found_date = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[3]/td[4]').text.strip() #成立日期
                company_loc = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[11]/td[2]').text.split('\n')[0].split(' ')[0] #企业地址'
                company_cover = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[12]/td[2]').text.strip() # 经营范围
                company_tax_num = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[4]/td[4]').text.strip() #'纳税人识别号'
                company_prov = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[8]/td[2]').text.strip()  #所属地区
                #print('公司名称：%s' %company_title)
                try:
                    info_0 = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[3]/td[2]').text.strip() #经营状态
                except:
                    info_0 = None
                if '在业' in info_0:
                    info = 1
                elif '存续' in info_0:
                    info = 2
                elif '注销' in info_0:
                    info = 10
                elif '吊销' in info_0:
                    info = 9
                elif '筹建' in info_0:
                    info = 3
                elif '清算' in info_0:
                    info = 4
                elif '迁入' in info_0:
                    info = 5
                elif '迁出' in info_0:
                    info = 6
                elif '停业' in info_0:
                    info = 7
                elif '撤销' in info_0:
                    info = 8
                elif '开业' in info_0:
                    info = 1
                else:
                    info = None
                msg = [company_prov, company_title, company_lead, found_date, found_cash, company_loc, company_email, company_tel,\
                       company_cover, search_url, company_tax_num, info]
                print('%s company is done.' %name)
                #print(msg)
                browser.quit()
                return msg
            except Exception as e:
                print(repr(e))
                print('企业网页数据提取出现问题')
                return 0

    def up_sql(self, data): #上传数据库dic_company_detail
        '''
        '公司名', '法人', '成立日期', '注册资本', '地址', '邮箱', '电话', '经营范围', '企查查主页', '纳税识别号', '公司id'
        '''
        connection = pymysql.connect(host = '39.105.9.20',user = 'root',password = 'bigdata_oil',db = 'cxd_data',charset = 'utf8')
        try :
            with connection.cursor() as cursor:
                sql_i = "insert into `dic_company_detail`(`name`,`person`,`date`,`capital`,`address`,`email`,`phone`,`operate`,`website`,\
                `tax_num`,`company_id`)\
                values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
                cursor.execute(sql_i,data)
            connection.commit() 
            #print('成功')
        except Exception as e:
            print(e)
        finally:
            connection.close()
        print('"dic_company_detail" Single is Done')

    def data_handle_txt(self, msg, name): #txt数据整理排序
        msg = msg[:-1] #去掉经营状态
        connection = pymysql.connect(host = '39.105.9.20',user = 'root',password = 'bigdata_oil',db = 'cxd_data',charset = 'utf8')

        sql_up = "select id from dic_company where name = '%s'"
        cursor = connection.cursor()
        cursor.execute(sql_up % name)
        c_id = cursor.fetchone()[0]
        msg = msg + [c_id,]
        #print(len(msg[1:]))
        return msg[1:]

    def get_txt(self,path,encoding='utf8'): #待更新数据从文本txt获取
        f = open(path,encoding=encoding)
        data_name = f.read()
        data_list = data_name.split('\n')
        return data_list

    def insert_c_sql(self, data): #新企业上传数据库dic_company
        connection = pymysql.connect(host = '39.105.9.20',user = 'root',password = 'bigdata_oil',db = 'cxd_data',charset = 'utf8')
        if '长兴岛' in data[5]:
            try :
                with connection.cursor() as cursor:
                    sql_i = "INSERT INTO `dic_company` (`name`,`com_group`,`com_type`,`company_province`,`company_provinceId`,`company_city`,`company_cityId`)\
                    VALUES (%s,%s,%s,%s,%s,%s,%s)"
                    cursor.execute(sql_i,[data[1],1,data[-1],'辽宁省',8,'大连市',51])
                connection.commit() 
                #print('成功')
            except Exception as e:
                print(e)
        else:
            try :
                with connection.cursor() as cursor:
                    sql_i = "INSERT INTO `dic_company` (`name`,`com_group`,`com_type`,`company_province`)\
                    VALUES (%s,%s,%s,%s)"
                    cursor.execute(sql_i,[data[1],0,data[-1],data[0]])
                connection.commit() 
                #print('成功')
            except Exception as e:
                print(e)
            finally:
                connection.close()
        print('"dic_company" Single is Done')
    

            
def main_txt(): #新增用
    q = QichachaSelenium()
    cookies = q.get_cookies()
    data = q.get_txt('需录入企查查企业2.txt')
#     data = ['中油澄丰石化（大连）有限公司', '大连华兴九洲石油化工有限公司', '大连金文汇创石油化工有限公司', '大连中屹石化有限公司']
    error_list = []
    for index in range(7636,len(data)): 
#     for index in range(0,len(data)):
        print('now index is:',index)        
        name = data[index]
        if name == '':
            print('txt空值')
            continue
        
        try:
            msg = q.get_company_info(name, cookies)
            print(msg)
            if msg == 0:
                error_list.append(name)
                time.sleep(random.uniform(1.4,4.4))
                print('已添加到不符企业名称')
                print(error_list)
                time.sleep(random.uniform(44,55.7))
                continue
            q.insert_c_sql(msg) #上传数据库dic_company
        except pymysql.err.IntegrityError:
            data_up = q.data_handle_txt(msg, name)
            #print(data_up)
            q.up_sql(data_up)
            #break
            continue
        try:
            data_up = q.data_handle_txt(msg, msg[1]) #这里要使用之前查的名字，因为不一定能在企查查上查到
            #print(data_up)
#             break            
            q.up_sql(data_up) #上传数据库dic_company_detail
            time.sleep(random.uniform(34,75.7))
        except Exception as e:
            print(str(e))
            print(name,'公司数据处理出现问题')
            error_list.append(name)
            print(error_list)
            return error_list
            break
        
        
        
if __name__ == '__main__':
    e_list = main_txt()
    print(e_list)

now index is: 7237
吉林省达美运输有限公司 company is done.
['', '吉林省达美运输有限公司', '', '', '600万元人民币', '', '553281287@qq.com', '13844411667', '', 'https://www.qichacha.com/firm_159647df33048d9b63dc4123c6681500.html', '', None]
"dic_company" Single is Done


D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'date' at row 1")
  result = self._query(query)


"dic_company_detail" Single is Done
now index is: 7238
盘锦拓达新石化贸易有限公司 company is done.
['辽宁省', '盘锦拓达新石化贸易有限公司', '陈威', '2019-05-20', '1000万元人民币', '辽宁省盘锦市盘山县古城子镇拉拉村', '暂无', '暂无', '销售:道路沥青、润滑油基础料、石油焦、有机热载体、针状焦、变压器油、沥青改性剂、润滑油、改性沥青、乳化沥青、工业白油、燃料油(闪点大于61℃)、化工产品(不含监控、易制毒及危险化学品)、蜡油、钢材、建材、五金(不含油漆)、煤炭、凡士林;批发(无储存)一般危险化学品类:石脑油、溶剂油[闭杯闪点≤60℃]、苯、1,2-二甲苯、1,3-二甲苯、1,4-二甲苯、甲醇、混合苯、丙烯、丙烷、环丙烷、正丁烷、异丁烷、戊烷、壬烷及其异构体、苯乙烯[稳定的]、癸烷、正辛烷、氢气、液化石油气、煤焦油、己烷、庚烷、1,3-丁二烯[稳定的]、乙醇[无水]、甲基叔丁基醚、1-戊烯、2-戊烯、异辛烷、汽油、柴油(闭杯闪点≤60℃)、煤油(凭危险化学品经营许可证有效期限经营);建筑劳务分包;货物装卸;货物搬运。(依法须经批准的项目,经相关部门批准后方可开展经营活动。)', 'https://www.qichacha.com/firm_574cbea71e8bbd023d8bccffea8dbc0b.html', '91211122MA0YNFLX82', 2]
"dic_company" Single is Done


D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'operate' at row 1")
  result = self._query(query)


"dic_company_detail" Single is Done
now index is: 7239
湖南长大建设集团股份有限公司 company is done.
['湖南省', '湖南长大建设集团股份有限公司', '何立伟', '1997-10-21', '60000万元人民币', '长沙市天心区芙蓉南路一段499号彩虹都家园办公楼1801-2001', '873349482@qq.com 更多邮箱', '0731-85412860', '建筑工程施工总承包或专业承包业务;建筑工程机械、架管、扣件、顶托的租赁;房地产开发经营。(依法须经批准的项目,经相关部门批准后方可开展经营活动)', 'https://www.qichacha.com/firm_75058397165f63e249d075b17236686a.html', '9143000071705054XQ', 2]
"dic_company" Single is Done
"dic_company_detail" Single is Done
now index is: 7240
邵阳公路桥梁建设有限责任公司 company is done.
['湖南省', '邵阳公路桥梁建设有限责任公司', '赵应华', '2001-01-05', '26000万元人民币', '邵阳市火车南站管理区铜铃社区铜铃西路38号', 'sylq168@126.com', '0739-5051058', '承包与其实力、规模、业绩相适应的国外工程项目,对外派遣实施上述境外工程所需的劳务人员。从事公路桥梁设计、施工、工程机械有偿服务、交通工程、路桥工程、工程机械维修及零配件供应及科技咨询、园林绿化、房屋建设(凭资质证书从事资质相适应的经营活动)。', 'https://www.qichacha.com/firm_b8e3550914d609359808c28f15bd4223.html', '9143050018553223XA', 2]
"dic_company" Single is Done
"dic_company_detail" Single is Done
now index is: 7241
湖南百舸水利建设股份有限公司 company is done.
['湖南省', '湖南百舸水利建设股份有限公司', '

...

"dic_company_detail" Single is Done
now index is: 7578
延边奕燊运输有限公司 company is done.
['吉林省', '延边奕燊运输有限公司', '张秀兰', '2018-09-20', '30万元人民币', '延吉市站前街300-3-10(妇幼医院斜对面)', 'mingxiangcaiwu@163.com', '13089314555', '道路普通货物运输;装卸搬运服务;沙、石加工;机械加工及制作;水电焊服务;机械设备、电器维修;建筑劳务分包;室内外装修;水电安装;批发兼零售:建筑材料(不含原木)、装饰材料、五金、交电、家用电器、电缆、办公用品、日用百货、劳保用品、消防器材;机电设备销售及租赁;电力设备拆除及销售(依法须经批准的项目,经相关部门批准后方可开展经营活动)', 'https://www.qichacha.com/firm_c729186a3f5ab3362afe9cd4c801294d.html', '91222401MA16XCDJ51', 2]
"dic_company" Single is Done
"dic_company_detail" Single is Done
now index is: 7579
中交一公局集团有限公司 company is done.
['北京市', '中交一公局集团有限公司', '都业洲', '1987-01-07', '587344.574475万元人民币', '北京市朝阳区管庄周家井', 'qyghb@cfhec.cn 更多邮箱', '010-65168093', '公路、桥梁、隧道、市政、房建、交通工程、铁路、机场、站场、车场和船坞的工程施工;工程总承包;工程设计与咨询;工程项目管理;技术研发;公路工程综合甲级工程试验检测(限分支机构经营);货物进出口、技术进出口、代理进出口;出租办公用房;出租商业用房。(企业依法自主选择经营项目,开展经营活动;依法须经批准的项目,经相关部门批准后依批准的内容开展经营活动;不得从事本市产业政策禁止和限制类项目的经营活动。)', 'https://www.qichacha.com/firm_80526a16847fb25cc792acf260fede11.html', '911100001017004524',

"dic_company_detail" Single is Done
now index is: 7594
北京汇银丰建筑工程有限公司 company is done.
['北京市', '北京汇银丰建筑工程有限公司', '王文彬', '2018-11-08', '6000万元人民币', '北京市平谷区大华山镇大华山大街269号', 'huiyinfeng2018@163.com', '010-84867217', '施工总承包、专业承包、劳务分包;机械设备租赁;园林绿化;园林景观艺术设计;建筑物清洁服务;工程项目管理;技术开发及转让、技术咨询与服务;经济贸易咨询;电脑图文设计、制作;承办展览展示;组织文化艺术交流活动;会议服务;物业管理;产品设计;设计、制作、代理、发布广告;销售建筑材料、装饰材料、计算机软硬件及外围设备、日用品、工艺品、机械设备、五金交电(不含电动自行车)、电子产品、金属制品、化工产品。(企业依法自主选择经营项目,开展经营活动;依法须经批准的项目,经相关部门批准后依批准的内容开展经营活动;不得从事本市产业政策禁止和限制类项目的经营活动。)', 'https://www.qichacha.com/firm_827adba4331910319abf26aee12fc230.html', '91110117MA01FGB118', 1]
"dic_company" Single is Done
"dic_company_detail" Single is Done
now index is: 7595
北京天一腾达物流有限公司 company is done.
['北京市', '北京天一腾达物流有限公司', '蔡开旺', '2016-06-02', '1000万元人民币', '北京市顺义区大孙各庄镇府前东街9号-276', '1584843457@qq.com', '13717672782 更多号码', '普通货运;货物专用运输(集装箱);国内道路货运代理;农产品仓储服务;包装服务;装卸服务。(企业依法自主选择经营项目,开展经营活动;依法须经批准的项目,经相关部门批准后依批准的内容开展经营活动;不得从事本市产业政策禁止和限制类项目的经营活动。)', 'https://www.qichacha.com/firm_58885d813c57318ad3cd6

"dic_company_detail" Single is Done
now index is: 7609
国储石化（海南）有限公司 company is done.
['海南省', '国储石化（海南）有限公司', '黄文志', '2018-04-26', '10000万元人民币', '海南国际旅游岛先行试验区文黎大道8号A座1413室', '13911071997@qq.com', '13911071997', '汽油、柴油、石油制品、燃料油、润滑油、天然气、液化气、硫磺、颗粒状硫磺、液体硫磺、粉状硫磺、重质油、航空煤油、原料油、乙二醇、轻循环油、凝析油、基础油、抽余油、液压油、碳四、碳五、稳点轻烃、混合芳烃、3#喷气燃料、生物柴油、化工原料及产品(以上产品不含专项审批及危险品)、矿产品、焦炭、煤炭、沥青、石蜡制品、五金机械、金属材料、仪器仪表、汽车配件、钢材、建筑材料、装潢材料、食用农产品的批发及销售;商务咨询(金融、证券、期货及资产类除外);白油、粗白油、管道运输设备批发;充电桩销售;贸易代理、技术进出口、货物进出口(国家限定公司经营或禁止进出口的商品和技术除外);环保设备批发;预包装食品批发。', 'https://www.qichacha.com/firm_bef6b5d559fa476684d98e4973fd8589.html', '91469034MA5T444W2R', 2]
"dic_company" Single is Done
"dic_company_detail" Single is Done
now index is: 7610
黑河市创世伟业危险品运输股份有限公司 company is done.
['黑龙江省', '黑河市创世伟业危险品运输股份有限公司', '徐文波', '2016-08-19', '500万元人民币', '黑河市爱辉区机场路车辆管理所对面(老友谊砖厂院内)', '462085396@qq.com 更多邮箱', '0456-6966666', '危险货物道路运输活动,建筑工程机械与设备租赁,砂石开采、销售,普通货物道路运输。', 'https://www.qichacha.com/firm_2c831b5daba9d918ad4de0400d3ffa14.html', '91231100MA18YRGA9T', 2

"dic_company_detail" Single is Done
now index is: 7625
山东百洲石化有限公司 company is done.
['山东省', '山东百洲石化有限公司', '路林鑫', '2017-07-13', '5000万元人民币', '山东省日照市东港迎宾路北、临沂路西(华地丽舍)001幢01单元01-303', '13863320809@126.com 更多邮箱', '0633-8012567', '汽油、柴油[闪点≤60℃]、煤油、石油气[液化的]、天然气[富含甲烷的]、二甲醚、1,3-丁二烯[稳定的]、丙烷、丙烯、二氧化碳[压缩的或液化的]、环氧乙烷、甲烷、正戊烷、正丁烷、硝化沥青、乙醇[无水]、正丁醇、正丁醛、正己烷、正辛烷、重质苯、3-氯丙烯、苯、苯乙烯[稳定的]、煤焦油、新型燃料油[闭杯闪点≤60℃]、石脑油、石油醚、石油原油、甲醇、甲基叔丁基醚、粗苯、二甲苯异构体混合物、1,3-二甲苯 、1,3-二氯丙烷、1,4-二甲苯 、2-氯丙烯、1,2-二甲苯 、1,2-二氯丙烷、1,2-环氧丙烷、3-氯-1,2-环氧丙烷、煤焦沥青、溶剂油[闭杯闪点≤60℃](不带有储存设施经营;天然气、石油气、丙烷、正丁烷限于工业生产原料等非燃料用途)(凭危险化学品经营许可证经营,有限期限以许可证为准);化工产品及化工原料批发零售(不含危险化学品、易制毒化学品、监控化学品及国家专项许可产品);有色金属、沥青(煤焦沥青,硝化沥青除外)、渣油、润滑油、煤炭(不含市区露天散煤)、焦炭、蜡油、五金建材、矿石、重油、铁粉、办公用品、洗化用品、农产品、机电产品、阀门批发零售;商务信息咨询;普通货物进出口;调和油品技术服务咨询(依法须经批准的项目,经相关部门批准后方可开展经营活动)。', 'https://www.qichacha.com/firm_890d36414be5e79385ed5e1149eca3ba.html', '91371102MA3F7UG910', 10]
"dic_company" Single is Done
"dic_company_detail" Single is Done
now index is: 7626
贵州有色地质安顺勘测院 company is done.
['贵州省', '贵州有色地质安顺勘测院', 

KeyboardInterrupt: 

In [27]:
cookie_test = cookie
print(cookie_test)

[{'domain': '.qichacha.com', 'expiry': 1607396978, 'httpOnly': False, 'name': 'Hm_lvt_3456bee468c83cc63fb5147f119f1075', 'path': '/', 'secure': False, 'value': '1575860979'}, {'domain': '.qichacha.com', 'expiry': 1591585778, 'httpOnly': False, 'name': 'UM_distinctid', 'path': '/', 'secure': False, 'value': '16ee8a242df12d-0f9bfa08c6d415-2393f61-1fa400-16ee8a242e0af3'}, {'domain': '.qichacha.com', 'httpOnly': False, 'name': 'Hm_lpvt_3456bee468c83cc63fb5147f119f1075', 'path': '/', 'secure': False, 'value': '1575860979'}, {'domain': '.qichacha.com', 'expiry': 1607396978, 'httpOnly': False, 'name': 'zg_did', 'path': '/', 'secure': False, 'value': '%7B%22did%22%3A%20%2216ee8a241e3a46-0cf961470bb314-2393f61-1fa400-16ee8a241e492f%22%7D'}, {'domain': '.qichacha.com', 'expiry': 1607396978, 'httpOnly': False, 'name': 'zg_de1d1a35bfa24ce29bbf2c7eb17e6c4f', 'path': '/', 'secure': False, 'value': '%7B%22sid%22%3A%201575860978151%2C%22updated%22%3A%201575860978151%2C%22info%22%3A%201575860978156%2C%

In [ ]:
['哈尔滨市王岗镇中心小学校', '茂名市茂南区羊角山禾贸易有限公司', '苏州五十铃汽车技术服务有限公司', '天津大学', '辽宁团团律师事务所', \
 '大连市人民政府办公室']

In [29]:
f= open('qichacha_cookies.txt')
data_json = f.read()
data_handle = json.loads(data_json)
print(type(data_handle))
print(data_handle)

<class 'list'>
[{'domain': '.qichacha.com', 'expiry': 1607396978, 'httpOnly': False, 'name': 'Hm_lvt_3456bee468c83cc63fb5147f119f1075', 'path': '/', 'secure': False, 'value': '1575860979'}, {'domain': '.qichacha.com', 'expiry': 1591585778, 'httpOnly': False, 'name': 'UM_distinctid', 'path': '/', 'secure': False, 'value': '16ee8a242df12d-0f9bfa08c6d415-2393f61-1fa400-16ee8a242e0af3'}, {'domain': '.qichacha.com', 'httpOnly': False, 'name': 'Hm_lpvt_3456bee468c83cc63fb5147f119f1075', 'path': '/', 'secure': False, 'value': '1575860979'}, {'domain': '.qichacha.com', 'expiry': 1607396978, 'httpOnly': False, 'name': 'zg_did', 'path': '/', 'secure': False, 'value': '%7B%22did%22%3A%20%2216ee8a241e3a46-0cf961470bb314-2393f61-1fa400-16ee8a241e492f%22%7D'}, {'domain': '.qichacha.com', 'expiry': 1607396978, 'httpOnly': False, 'name': 'zg_de1d1a35bfa24ce29bbf2c7eb17e6c4f', 'path': '/', 'secure': False, 'value': '%7B%22sid%22%3A%201575860978151%2C%22updated%22%3A%201575860978151%2C%22info%22%3A%2015

In [11]:
url = 'https://www.qichacha.com/cbase_a700673c1a64195944db0092fe0f78bb.html' #目标网址
browser = webdriver.Chrome()
url_l= 'https://www.qichacha.com/user_login' #这一步特别重要，必须先打开一个目标网址对应的界面
browser.get(url_l) 
for cookie_item in data_handle:
    #print({'name': cookie_item, 'value': cookies[cookie_item]})
    browser.add_cookie(data_handle)
browser.get(url)

In [44]:
q = QichachaSelenium()
cookies = q.get_cookies()

name = '内蒙古隆正煤炭运销有限公司'

url_search = 'https://www.qichacha.com/search?key=%s' %name #目标网址
browser = webdriver.Chrome()
url_l= 'https://www.qichacha.com/user_login' #这一步特别重要，必须先打开一个目标网址对应的界面
browser.get(url_l) 
for cookie_item in cookies:
    #print({'name': cookie_item, 'value': cookies[cookie_item]})
    browser.add_cookie(cookie_item)
browser.get(url_search)

try:
    num = browser.find_element_by_xpath('//*[@id="countOld"]/span').text
except Exception as e:
    print('基本被封，请手动查询')
    print(str(e))
    #return None

if num.strip() == '0':
    print('无此企业', name)
#     return 0
else:
    time.sleep(1)
    search_url = browser.find_element_by_xpath('//*[@id="search-result"]/tr[1]/td[3]/a').get_attribute("href") 
    url_c = 'https://www.qichacha.com%s' %'/cbase'+search_url[29:-5]
    #print(url_c)
    #com_web_q = 'https://www.qichacha.com' + search_url #企查查的企业主页
    url_company = browser.get(url_c) 
    time.sleep(1)
    button = browser.find_element_by_xpath('//*[@id="firstepdadModal"]/div/div/div[2]/button')
    browser.execute_script("arguments[0].click();", button)
    time.sleep(1)
    #table =  browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody').text
    #print(table)
    company_title = browser.find_element_by_xpath('//*[@id="company-top"]/div[2]/div[2]/div[1]/h1').text #企业名
    company_lead = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[1]/td[2]/div/div/div[2]/a[1]/h2').text #法人
    company_tel = browser.find_element_by_xpath('//*[@id="company-top"]/div[2]/div[2]/div[3]/div[1]/span[1]/span[2]').text.strip()
    company_email = browser.find_element_by_xpath('//*[@id="company-top"]/div[2]/div[2]/div[3]/div[2]/span[1]/span[2]').text.strip()
    #print(company_lead, company_tel, company_email)

    found_cash = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[1]/td[4]').text.strip() #注册资本
    found_date = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[3]/td[4]').text.strip() #成立日期
    company_loc = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[11]/td[2]').text.split('\n')[0] #企业地址'
    company_cover = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[12]/td[2]').text.strip() # 经营范围
    company_tax_num = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[4]/td[4]').text.strip() #'纳税人识别号'
    company_prov = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[8]/td[2]').text.strip()  #所属地区
    #print('公司名称：%s' %company_title)
    try:
        info_0 = browser.find_element_by_xpath('//*[@id="Cominfo"]/table/tbody/tr[3]/td[2]').text.strip() #经营状态
    except:
        info_0 = None
    if '在业' in info_0:
        info = 1
    elif '存续' in info_0:
        info = 2
    elif '注销' in info_0:
        info = 10
    elif '吊销' in info_0:
        info = 9
    elif '筹建' in info_0:
        info = 3
    elif '清算' in info_0:
        info = 4
    elif '迁入' in info_0:
        info = 5
    elif '迁出' in info_0:
        info = 6
    elif '停业' in info_0:
        info = 7
    elif '撤销' in info_0:
        info = 8
    elif '开业' in info_0:
        info = 1
    else:
        info = None
    msg = [company_prov, company_title, company_lead, found_date, found_cash, company_loc, company_email, company_tel,\
           company_cover, search_url, company_tax_num, info]
    print('%s company is done.' %name)
    print(msg)
    

内蒙古隆正煤炭运销有限公司 company is done.
['内蒙古自治区', '内蒙古隆正煤炭运销有限公司', '吕平', '2017-07-26', '200万元人民币', '准格尔旗暖水乡暖水村 查看地图 附近企业', '暂无', '15149733666', '许可经营项目:普通货物运输。 一般经营项目:煤炭、润滑油、轮胎、汽车配件销售', 'https://www.qichacha.com/firm_387139517b515ada98daffb9d810b4e6.html', '91150622MA0NFJ1AXM', 2]


In [50]:
company_loc.split(' ')[0]

'准格尔旗暖水乡暖水村'

In [ ]:
//*[@id="Cominfo"]/table/tbody/tr[3]/td[2]
//*[@id="Cominfo"]/table/tbody/tr[3]/td[2]